In [13]:
import numpy as np
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers import normalization
import h5py
from test import *
from keras import backend as K
import time
from FaceRec.pretrained_cnn import *
import FaceRec
from FaceRec.net import *
from FaceRec.get_input import *
from keras.utils import np_utils

x,y = from_file()
x_test, y_test = test_file()

model = Sequential()
model.add(Convolution2D(64, 3, 3,input_shape=(3, 227, 227), activation='relu', name='conv1_1', border_mode='same'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [14]:
cnn = pretrained_cnn()

for k in cnn[cnn.keys()[0]]:
    for i in k:
        a = i[0][0][1][0]
        if 'conv' in a:
            weight1 = np.rollaxis(i[0][0][2][0][0],3,start=0)
            weight1 = np.rollaxis(weight1,3,start=1)
            weight2 = np.rollaxis(i[0][0][2][0][1],1,start=0)[0]
            weights = [weight1,weight2]
            layer_dict[a].set_weights(weights)
            print "Weights added to",a

In [15]:
model.add(Flatten())
model.add(Dense(output_dim=4096, activation='relu', init="uniform"))
model.add(Dense(output_dim=4096, init="uniform", activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim=21, init="uniform", activation='softmax'))

In [16]:
x,y = from_file()
x_test, y_test = test_file()

In [10]:
x = x.astype('float32')
x_test = x_test.astype('float32')
x /= 255
x_test /= 255
# x = np.subtract(x,np.average(x))
# x_test = np.subtract(x_test,np.average(x))
y = np_utils.to_categorical(y, 21)
y_test = np_utils.to_categorical(y_test, 21)

In [11]:
sgd = SGD(lr=0.01, decay=5e-4, momentum=0.9, nesterov=True)
# adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
print "Running training"
for i in xrange(10):
    model.fit(x, y, nb_epoch=1, batch_size=1, verbose=1, show_accuracy=True, shuffle=True)
    print model.evaluate(x_test, y_test, batch_size=1, show_accuracy=True)
    print model.predict(x_test, batch_size=1)

KeyboardInterrupt: 

In [ ]:
x

In [ ]:
x_test